## Deliverable 2. Create a Customer Travel Destinations Map.

In [51]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [55]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Panaba,MX,21.2833,-88.2667,95.20,35,100,16.67,overcast clouds
1,1,Tasiilaq,GL,65.6145,-37.6368,35.76,86,100,1.52,overcast clouds
2,2,Skibotn,NO,69.3913,20.2679,51.03,72,9,14.54,clear sky
3,3,Jardim,BR,-21.4803,-56.1381,77.70,40,3,2.98,clear sky
4,4,Ponta Do Sol,PT,32.6667,-17.1000,67.71,71,100,4.83,overcast clouds
5,5,Mahebourg,MU,-20.4081,57.7000,71.26,94,40,3.44,scattered clouds
6,6,Vardo,NO,70.3705,31.1107,39.18,93,100,13.80,overcast clouds
7,7,Ambilobe,MG,-13.2000,49.0500,69.64,92,7,3.00,clear sky
8,8,Quetigny,FR,47.3167,5.1167,62.26,55,0,6.91,clear sky
9,9,Kincardine,GB,56.0667,-3.7167,51.93,78,7,12.62,clear sky


In [56]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 85


In [57]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
hotel_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
hotel_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,San Miguel,SV,13.4833,-88.1833,84.24,74,97,3.78,overcast clouds
12,12,Hithadhoo,MV,-0.6000,73.0833,83.08,65,100,3.74,overcast clouds
38,38,San Pedro,PH,14.3500,121.0167,81.36,90,83,4.18,broken clouds
45,45,Ormara,PK,25.2088,64.6357,81.54,89,5,9.55,clear sky
60,60,Cayenne,GF,4.9333,-52.3333,84.24,17,0,9.22,clear sky
88,88,Guntakal,IN,15.1667,77.3833,84.54,59,98,12.30,overcast clouds
96,96,Guanica,PR,17.9716,-66.9080,83.21,80,53,13.27,broken clouds
131,131,Kavaratti,IN,10.5669,72.6420,83.17,78,54,11.92,broken clouds
144,144,Mazamari,PE,-11.3258,-74.5308,80.74,66,91,0.76,overcast clouds
145,145,Rameswaram,IN,9.2833,79.3000,83.12,82,35,26.13,scattered clouds


In [58]:
# 4a. Determine if there are any empty rows.
hotel_df.count()

City_ID                81
City                   81
Country                81
Lat                    81
Lng                    81
Max Temp               81
Humidity               81
Cloudiness             81
Wind Speed             81
Current Description    81
dtype: int64

In [60]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = hotel_df.dropna()

In [70]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,San Miguel,SV,84.24,overcast clouds,13.4833,-88.1833,
12,Hithadhoo,MV,83.08,overcast clouds,-0.6000,73.0833,
38,San Pedro,PH,81.36,broken clouds,14.3500,121.0167,
45,Ormara,PK,81.54,clear sky,25.2088,64.6357,
60,Cayenne,GF,84.24,clear sky,4.9333,-52.3333,


In [71]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [72]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,San Miguel,SV,84.24,overcast clouds,13.4833,-88.1833,Excalibur
12,Hithadhoo,MV,83.08,overcast clouds,-0.6000,73.0833,Scoop Guest House
38,San Pedro,PH,81.36,broken clouds,14.3500,121.0167,Esquivel's Garden And Resort
45,Ormara,PK,81.54,clear sky,25.2088,64.6357,Kenara Colony house for Rent
60,Cayenne,GF,84.24,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
88,Guntakal,IN,84.54,overcast clouds,15.1667,77.3833,Vasavi Pipes Office
96,Guanica,PR,83.21,broken clouds,17.9716,-66.9080,Copamarina Beach Resort & Spa
131,Kavaratti,IN,83.17,broken clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
144,Mazamari,PE,80.74,overcast clouds,-11.3258,-74.5308,Hospedaje Ratto
145,Rameswaram,IN,83.12,scattered clouds,9.2833,79.3000,Daiwik Hotels Rameswaram


In [73]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [74]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weahter Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [77]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))